In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("../input/tipping/tips.csv")
df.head(10)

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
correlation = df.corr()
correlation

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
sns.heatmap(correlation)
plt.show()

In [ ]:
sns.catplot(x="sex",data=df,kind="count")
plt.show()

In [ ]:
sns.catplot(x="sex",y="tip",data=df,kind="box")
plt.show()

In [ ]:
df.day.unique()

In [ ]:
sns.relplot(x="total_bill",y="tip",data=df,kind="scatter",hue="day")
plt.show()

In [ ]:

sns.catplot(x="time",data=df,kind="count")
plt.show

In [ ]:
df.corrwith(df["total_bill"])

In [ ]:
df.head()

In [ ]:
columns_to_encode = ["sex","day","time"]
columns_to_scale = ["total_bill","tip","size"]

In [ ]:
df = pd.concat([df, pd.get_dummies(df["sex"],prefix="sex")], axis=1)
df = pd.concat([df, pd.get_dummies(df["day"],prefix="day")], axis=1)
df = pd.concat([df, pd.get_dummies(df["time"],prefix="time")], axis=1)
df = pd.concat([df, pd.get_dummies(df["smoker"],prefix="smoker")], axis=1)
df.head()

In [ ]:
df.drop(["sex","smoker","day","time"], axis=1, inplace=True)
df.head()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_columns = pd.DataFrame(scaler.fit_transform(df[columns_to_scale]),columns=columns_to_scale)
scaled_columns.describe()

In [ ]:
df.drop(["total_bill","tip","size","sex_Male","smoker_No","time_Dinner"],axis=1,inplace=True)
df = pd.concat([df, scaled_columns], axis=1)

In [ ]:
correlation = df.corr().abs()
plt.figure(figsize=(8,8))
sns.heatmap(correlation, annot=True)
plt.show()

In [ ]:
target = df.tip
features = df.loc[:,df.columns!="tip"]

In [ ]:
from sklearn.feature_selection import SelectKBest
kbest = SelectKBest(k=5)
k_best_features = kbest.fit_transform(features, target)
list(df.columns[kbest.get_support (indices=True)])

In [ ]:
df.corrwith(df["tip"]).abs().nlargest(6)

In [ ]:
X = df[["total_bill","tip","day_Sun","day_Sat"]]
Y = df["sex_Female"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
knn = KNeighborsClassifier()
param_grid = [{"n_neighbors":[3, 5, 9, 13, 17, 19], "weights":["uniform","distance"]}]
grid_search = GridSearchCV(knn, param_grid, cv=10, scoring="accuracy")
grid_search.fit(X, Y)
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
from sklearn.svm import SVC
svclassifier = SVC()
svclassifier.fit(X_train, y_train)
predsvc = svclassifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predsvc, y_test))

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(svclassifier, X, Y, cv=10).mean()

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)
scores = cross_val_score(svclassifier, X, Y, cv=skf)
print("skorlar:\n{}".format(scores))
print("skorların ortalaması:\n{}".format(scores.mean()))

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [ {"C":[0.0001, 0.01, 0.1, 1, 10], "kernel":["poly","linear","rbf"],"gamma":[0.0001, 0.01, 0.1, 1, 10]}]
grid_search = GridSearchCV(svclassifier, param_grid, cv=5, scoring="accuracy")
grid_search.fit(X, Y)
grid_search.best_estimator_

In [ ]:
grid_search.best_score_

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=21, weights="uniform")
knn.fit(X_train, y_train)
predknn = knn.predict(X_test)
print(classification_report(y_test, predknn))

In [ ]:
X = df[["sex_Female","tip","day_Sun","day_Sat"]]
Y = df["total_bill"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

In [ ]:
from sklearn.linear_model import LinearRegression
linearreg = LinearRegression()
linearreg.fit(X_train, y_train)
pred = linearreg.predict(X_test)

In [ ]:
print("Coefficients: \n", linearreg.coef_)
print("Intercept: \n", linearreg.intercept_)

In [ ]:
X = df[["total_bill"]]
y = df.tip

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
linearreg = LinearRegression()
linearreg.fit(X_train, y_train)

In [ ]:
pred = linearreg.predict(X_test)
from sklearn.metrics import r2_score
print(r2_score(pred, y_test))

In [ ]:
import matplotlib.pyplot as plt
fig,ax=plt.subplots()
plt.scatter(X_test, y_test, color="black")
plt.plot(X_test, pred, color="red",linewidth=3)
plt.show()

In [ ]:
print("Coefficients: \n", linearreg.coef_)
print("Intercept: \n", linearreg.intercept_)

In [ ]:
X = df[["total_bill","day_Sun","day_Sat","tip","day_Thur","day_Fri"]]
Y = df["sex_Female"]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y)

In [ ]:
from sklearn.svm import SVC
svclassifier = SVC(C=0.0001, kernel="poly",degree=6)
svclassifier.fit(X_train, y_train)
predsvc = svclassifier.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, predsvc))

Cross validation

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(svclassifier, X, Y, cv=5).mean()

Shuffle Split Cross Validation

In [ ]:
from sklearn.model_selection import ShuffleSplit
shuffle_split = ShuffleSplit(test_size=0.5, train_size=0.5, n_splits=10)
scores = cross_val_score(svclassifier, X, Y, cv=shuffle_split)
print("skorlar:\n{}".format(scores))
print("skorların ortalaması:\n{}".format(scores.mean()))

Stratified KFold

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)
scores = cross_val_score(svclassifier, X, Y, cv=skf)
print("skorlar:\n{}".format(scores))
print("skorların ortalaması:\n{}".format(scores.mean()))

Grid Search

In [ ]:
best_score = 0
for C in [0.0001, 0.001, 0.1, 1, 10, 100]:
    for kernel in ["poly","linear","rbf"]:
        svm = SVC(C=C, kernel=kernel)
        svm.fit(X_train, y_train)
        score = svm.score(X_test, y_test)
        if score > best_score:
            best_score = score
            best_parameters = {"C":C,"kernel":kernel}
print("Best score:{:.2f}".format(best_score))
print("Best parameters: {}".format(best_parameters))

In [ ]:
#önce training+validation set ve test sete böldük
X_trainval, X_test, y_trainval, y_test = train_test_split(X, Y, test_size=0.1)
#önceki setten training seti ve validation seti çektik
X_train, X_valid, y_train, y_valid = train_test_split(X_trainval, y_trainval, test_size=0.1)
best_score = 0
for C in [0.0001, 0.001, 0.1, 1, 10, 100]:
    for kernel in ["poly","linear","rbf"]:
        svm = SVC(C=C, kernel=kernel)
        svm.fit(X_train, y_train)
        score = svm.score(X_valid, y_valid)
        if score > best_score:
            best_score = score
            best_parameters = {"C":C,"kernel":kernel}
# bulduğumuz parametrelerle model yapıp test setiyle test edicez
svm = SVC(**best_parameters)
svm.fit(X_trainval, y_trainval)
test_score = svm.score(X_test, y_test)
print("Best score:{:.2f}".format(best_score))
print("Best parameters: {}".format(best_parameters))
print("Test setinde karşılaşılan skor: {:.2f}".format(test_score))

Grid Search Cross Validation

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [ {"C":[0.0001, 0.01, 0.1, 1, 10], "kernel":["poly","linear","rbf"],"gamma":[0.0001, 0.01, 0.1, 1, 10]}]
grid_search = GridSearchCV(svc, param_grid, cv=5, scoring="accuracy")
grid_search.fit(X, Y)
grid_search.best_estimator_

In [ ]:
print("En iyi parametreler: {}".format(grid_search.best_params_))

In [ ]:
print("En iyi CV skoru: {:.2f}".format(grid_search.best_score_))

Random Search CV

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
distributions = dict(C=uniform(loc=0, scale=4), kernel=["poly","rbf","linear"])
random = RandomizedSearchCV(svclassifier, distributions, random_state=0)
search = random.fit(X, Y)
search.best_params_

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train, y_train)
predknn = knn.predict(X_test)
print(accuracy_score(y_test, predknn))

In [ ]:
X = df[["total_bill","day_Sun","day_Sat","tip","day_Thur","day_Fri"]]
Y = df["sex_Female"]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, stratify=Y)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
dtree = DecisionTreeClassifier(criterion="entropy", max_depth=8)
dtree.fit(X_train, y_train)
preddt = dtree.predict(X_test)
print(accuracy_score(y_test,preddt))

In [ ]:
from sklearn.tree import export_graphviz
export_graphviz(dtree, out_file="tree3.png", class_names=["female","male"], feature_names=X.columns)

.dot dosyalarını açmak için: dot -Tpng DocName.dot -o DocName.png

Rassal orman

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier()
forest.fit(X_train, y_train)
predforest = forest.predict(X_test)
print(accuracy_score(y_test,predforest))

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = [{"n_estimators":[5, 10, 50],"criterion":["gini","entropy"],"max_depth":[4, 6, 8, 10]}]
grid_search = GridSearchCV(forest,param_grid,cv=10,scoring="accuracy")
grid_search.fit(X,Y)
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
xgbclassifier=XGBClassifier(silent=0,objective='reg:logistic',eval_metric="logloss")
xgbclassifier.fit(X_train, y_train)
predxgb = xgbclassifier.predict(X_test)
print(classification_report(y_test,predxgb,digits=2))
print(accuracy_score(y_test, predxgb))

In [ ]:
param_grid = [{"n_estimators":[5, 10, 50],"learning_rate":[0.001, 0.01, 0.1],"max_depth":[3, 5, 10],"gamma":[0.001, 0.01, 0.1, 1]}]
grid_search = GridSearchCV(xgbclassifier,param_grid,cv=5,scoring="accuracy")
grid_search.fit(X,Y)
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
from tpot import TPOTClassifier
from tpot import TPOTRegressor

tpot = TPOTClassifier(generations=5,verbosity=2)

tpot.fit(X_train,y_train)

In [ ]:
tpot.score(X_test, y_test)

In [ ]:
tpot.export('tpot_pipeline.py')

In [ ]:

#df = pd.concat([df, scaled_columns], axis=1)
#df.head()

In [ ]:
#df = pd.read_csv("/kaggle/input/tipping/tips.csv")
#df.head()

In [ ]:
#from sklearn.preprocessing import LabelEncoder
#le = LabelEncoder()
#df["day"] = le.fit_transform(df.day)
#df["smoker"] = le.fit_transform(df.smoker)
#df.head()

In [ ]:
#from sklearn.preprocessing import OneHotEncoder
#ohe = OneHotEncoder(sparse=False)
#ohe_encoded = ohe.fit_transform(df[columns_to_encode])
#ohe_encoded